<a href="https://colab.research.google.com/github/KabeloPhiri/Amercican-Express---Default-prediction/blob/main/AMEX_Default_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Colab Set Up

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kabelophiri","key":"28e855ce77305b43c53dbc27dc3ac1e7"}'}

In [ ]:
! mkdir ~/.kaggle


In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c amex-default-prediction

100% 20.5G/20.5G [03:01<00:00, 145MB/s]
100% 20.5G/20.5G [03:01<00:00, 121MB/s]


### Functions


In [ ]:
import numpy as np
import sympy as sp

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# importing classifiers to try with
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn import metrics
# importing train_test_split for train and validation split
from sklearn.model_selection import train_test_split
# importing SelectFromModel to select features from model 
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2, f_regression

from imblearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline as mp
# importing FeatureUnion for combining transformers
from sklearn.pipeline import FeatureUnion

# importing SimpleImputer for handling missing value
from sklearn.impute import SimpleImputer
# importing MissingIndicator for handling missing value
from sklearn.impute import MissingIndicator
# importing StandardScaler for standardization
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# importing OnHotEncoder for encoding categorical variable
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
# importing for transformation
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

# importing samplers for handling data imbalance
from imblearn.combine import SMOTEENN 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler 
from imblearn.under_sampling import RandomUnderSampler 

# importing metrics required for model evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# importing RepeatedKFold for cross validation
from sklearn.model_selection import RepeatedKFold
# importing for model evaluation
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
# importing RepeatedStratifiedKFold for model evaluation
from sklearn.model_selection import RepeatedStratifiedKFold,StratifiedKFold
# importing GridSearchCV for hyperparameter tuning
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV
from yellowbrick.model_selection import ValidationCurve
import itertools
from scipy.stats import randint as sp_randInt
from scipy.stats import uniform as sp_randFloat
from scipy.stats import uniform as sp_uniform


import time
pd.options.mode.chained_assignment = None  # default='warn'


### Data Import

There is too many data_samples to ingest and work in through collad. One good sampling theory that can work is the 10 times rule. (ie let number of sample be 10*number of columns/features)

In this case we can use more than 10 times. We will use 100 000 samples

In [ ]:
!unzip /content/amex-default-prediction.zip
train_data = pd.read_csv("/content/train_data.csv", nrows = 100000 )
test_data = pd.read_csv("/content/train_labels.csv", nrows =100000)
labels = pd.read_csv("/content/train_labels.csv",nrows = 100000)

Archive:  /content/amex-default-prediction.zip
  inflating: sample_submission.csv   
  inflating: test_data.csv           
  inflating: train_data.csv          
  inflating: train_labels.csv        


In [ ]:
print ("Number of training samples:", train_data.shape[0])
print ("Number of training features:", train_data.shape[1])

Number of training samples: 100000
Number of training features: 190


Merge training data to their labels


In [ ]:
train = pd.merge(train_data,labels,how = 'inner', on = ["customer_ID"])
test = test_data

In [ ]:
pd.set_option('max_columns', None)
train.head()


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,NaN,NaN,0.000630,0.080986,0.708906,0.170600,0.006204,0.358587,0.525351,0.255736,NaN,0.063902,0.059416,0.006466,0.148698,1.335856,0.008207,0.001423,0.207334,0.736463,0.096219,NaN,0.023381,0.002768,0.008322,1.001519,0.008298,0.161345,0.148266,0.922998,0.354596,0.152025,0.118075,0.001882,0.158612,0.065728,0.018385,0.063646,0.199617,0.308233,0.016361,0.401619,0.091071,CR,O,0.007126,0.007665,NaN,0.652984,0.008520,NaN,0.004730,6.0,0.272008,0.008363,0.515222,0.002644,0.009013,0.004808,0.008342,0.119403,0.004802,0.108271,0.050882,NaN,0.007554,0.080422,0.069067,NaN,0.004327,0.007562,NaN,0.007729,0.000272,0.001576,0.004239,0.001434,NaN,0.002271,0.004061,0.007121,0.002456,0.002310,0.003532,0.506612,0.008033,1.009825,0.084683,0.003820,0.007043,0.000438,0.006452,0.000830,0.005055,NaN,0.0,0.005720,0.007084,NaN,0.000198,0.008907,NaN,1,0.002537,0.005177,0.006626,0.009705,0.007782,0.002450,1.001101,0.002665,0.007479,0.006893,1.503673,1.006133,0.003569,0.008871,0.003950,0.003647,0.004950,0.894090,0.135561,0.911191,0.974539,0.001243,0.766688,1.008691,1.004587,0.893734,NaN,0.670041,0.009968,0.004572,NaN,1.008949,2.0,NaN,0.004326,NaN,NaN,NaN,1.007336,0.210060,0.676922,0.007871,1.0,0.238250,0.0,4.0,0.232120,0.236266,0.0,0.702280,0.434345,0.003057,0.686516,0.008740,1.0,1.003319,1.007819,1.000080,0.006805,NaN,0.002052,0.005972,NaN,0.004345,0.001535,NaN,NaN,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,NaN,NaN,0.002526,0.069419,0.712795,0.113239,0.006206,0.353630,0.521311,0.223329,NaN,0.065261,0.057744,0.001614,0.149723,1.339794,0.008373,0.001984,0.202778,0.720886,0.099804,NaN,0.030599,0.002749,0.002482,1.009033,0.005136,0.140951,0.143530,0.919414,0.326757,0.156201,0.118737,0.001610,0.148459,0.093935,0.013035,0.065501,0.151387,0.265026,0.017688,0.406326,0.086805,CR,O,0.002413,0.007148,NaN,0.647093,0.002238,NaN,0.003879,6.0,0.188970,0.004030,0.509048,0.004193,0.007842,0.001283,0.006524,0.140611,0.000094,0.101018,0.040469,NaN,0.004832,0.081413,0.074166,NaN,0.004203,0.005304,NaN,0.001864,0.000979,0.009896,0.007597,0.000509,NaN,0.009810,0.000127,0.005966,0.000395,0.001327,0.007773,0.500855,0.000760,1.009461,0.081843,0.000347,0.007789,0.004311,0.002332,0.009469,0.003753,NaN,0.0,0.007584,0.006677,NaN,0.001142,0.005907,NaN,1,0.008427,0.008979,0.001854,0.009924,0.005987,0.002247,1.006779,0.002508,0.006827,0.002837,1.503577,1.005791,0.000571,0.000391,0.008351,0.008850,0.003180,0.902135,0.136333,0.919876,0.975624,0.004561,0.786007,1.000084,1.004118,0.906841,NaN,0.668647,0.003921,0.004654,NaN,1.003205,2.0,NaN,0.008707,NaN,NaN,NaN,1.007653,0.184093,0.822281,0.003444,1.0,0.247217,0.0,4.0,0.243532,0.241885,0.0,0.707017,0.430501,0.001306,0.686414,0.000755,1.0,1.008394,1.004333,1.008344,0.004407,NaN,0.001034,0.0048

# EDA - Exploratory Data Anaysis

Statistical summary of data

In [ ]:
train.describe() 

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
count,99214.000000,1.000000e+05,100000.000000,99963.000000,1.000000e+05,81374.000000,9.996300e+04,9.996300e+04,14614.000000,6.980500e+04,9.494700e+04,1.000000e+05,99963.000000,1.000000e+05,1.000000e+05,78158.000000,100000.000000,87085.000000,10238.000000,99996.000000,100000.000000,9.955300e+04,43672.000000,1.000000e+05,1.000000e+05,1.000000e+05,99479.000000,94589.000000,100000.000000,2.675600e+04,1.000000e+05,1.000000e+05,1.000000e+05,99963.000000,1.000000e+05,81374.000000,100000.000000,1.000000e+05,96698.000000,45877.000000,9.909800e+04,1.000000e+05,1.000000e+05,47266.000000,100000.000000,98081.000000,100000.000000,89351.000000,99880.000000,100000.000000,86235.000000,1.000000e+05,9.996300e+04,4.380900e+04,100000.000000,9.996300e+04,11077.000000,9.996300e+04,96012.000000,100000.000000,1.000000e+05,1.000000e+05,1.000000e+05,9.645400e+04,9.996300e+04,9.830100e+04,100000.000000,9.958800e+04,100000.000000,100000.000000,1149.000000,1.000000e+05,9.963200e+04,1.000000e+05,11276.000000,1.000000e+05,1.000000e+05,54332.000000,99880.000000,9.996300e+04,9.494700e+04,9.864800e+04,1.000000e+05,5.793000e+03,1.000000e+05,9.963200e+04,1.000000e+05,1.000000e+05,9.996300e+04,9.953400e+04,2.712600e+04,1.000000e+05,100000.000000,100000.000000,1.000000e+05,9.645400e+04,1.000000e+05,1.000000e+05,9.947900e+04,1.000000e+05,7.072000e+03,99963.000000,1.000000e+05,1.000000e+05,73.0,1.000000e+05,1.000000e+05,150.000000,100000.000000,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,9.999500e+04,1.000000e+05,9.996300e+04,9.947900e+04,1.000000e+05,1.000000e+05,9.717300e+04,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,99672.000000,99995.000000,99677.000000,99792.000000,9.997900e+04,9.927400e+04,9.818700e+04,9.818700e+04,45858.000000,1.013000e+04,9.818700e+04,1.000000e+05,100000.000000,10875.000000,97616.000000,99963.000000,530.000000,9.995800e+04,634.000000,634.000000,673.000000,99942.000000,1.000000e+05,7.462100e+04,9.678400e+04,96784.000000,96784.000000,96784.000000,96784.000000,96784.000000,96784.000000,96784.000000,96784.000000,96784.000000,9.678400e+04,96784.000000,9.678400e+04,97869.000000,1.000000e+05,9.818700e+04,9.818700e+04,9.997900e+04,1355.000000,9.818700e+04,9.818700e+04,10183.000000,9.925000e+04,1.000000e+05,3616.000000,3616.000000,3616.000000,3616.000000,3616.000000,9.818700e+04,9.927400e+04,9.818700e+04,17272.000000,9.818700e+04,9.926700e+04,9.818700e+04,100000.000000
mean,0.653361,1.549806e-01,0.126169,0.615752,8.096814e-02,0.226610,6.344809e-02,1.339622e-01,0.183178,1.569983e-01,1.216858e-01,1.752239e-01,0.251915,8.169071e-02,4.979998e-02,0.473235,0.408267,0.387452,0.183977,0.145408,0.188232,4.509766e-01,0.167523,1.488764e-01,1.915225e-01,1.220394e-01,0.180635,0.600251,0.221938,7.492337e-02,8.530926e-02,1.132266e-01,2.465027e-01,0.982416,3.314527e-02,0.223939,0.092636,3.283848e-01,0.302685,0.205202,1.011405e-01,3.756728e-02,2.178217e-01,0.076268,0.105047,0.364994,0.376133,0.435253,0.055953,0.378103,0.189365,4.050961e-02,3.535676e-01,7.199943e-01,0.590366,1.538463e-01,0.

Training data consists of float64,int64 and object data types

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Columns: 191 entries, customer_ID to target
dtypes: float64(185), int64(2), object(4)
memory usage: 146.5+ MB


#### Dealing with missing values

In [ ]:
missing = train.isnull().sum()
print(missing.sort_values(ascending = False))

D_87      99927
D_88      99850
D_108     99470
D_111     99366
D_110     99366
          ...  
R_7           0
B_24          0
D_75          0
P_4           0
target        0
Length: 191, dtype: int64


In [ ]:
train_new = train.dropna(axis=1,how = 'any')


In [ ]:
train_new.head()

,customer_ID,S_2,D_39,B_1,R_1,B_4,B_5,R_2,D_47,B_7,D_51,B_9,R_3,B_10,S_5,B_11,S_6,R_4,B_12,S_8,R_5,D_58,B_14,D_60,S_11,D_63,D_65,B_18,S_12,R_6,S_13,B_21,D_71,S_15,B_23,P_4,D_75,B_24,R_7,R_8,S_16,R_10,R_11,S_17,R_12,B_28,R_13,R_14,R_15,R_16,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_21,R_22,R_23,D_92,D_93,D_94,R_24,R_25,D_96,B_36,B_37,B_40,D_127,R_28,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.001733,0.008724,0.009228,0.080986,0.170600,0.006204,0.525351,0.059416,1.335856,0.008207,0.001423,0.096219,0.023381,0.002768,0.008322,0.008298,0.148266,0.922998,0.001882,0.158612,0.018385,0.199617,0.401619,CR,0.007126,0.652984,0.272008,0.008363,0.515222,0.002644,0.119403,0.108271,0.050882,0.007554,0.069067,0.004327,0.007562,0.001434,0.002271,0.007121,0.002456,0.008033,1.009825,0.084683,0.003820,0.000438,0.006452,0.005055,0.005720,0.007084,0.000198,0.008907,1,0.002537,0.005177,0.006626,0.009705,0.002450,0.007479,0.006893,1.006133,0.003569,0.008871,0.003950,0.003647,0.004950,0.009968,0.004572,0.210060,1.003319,0.001535,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.005775,0.004923,0.006151,0.069419,0.113239,0.006206,0.521311,0.057744,1.339794,0.008373,0.001984,0.099804,0.030599,0.002749,0.002482,0.005136,0.143530,0.919414,0.001610,0.148459,0.013035,0.151387,0.406326,CR,0.002413,0.647093,0.188970,0.004030,0.509048,0.004193,0.140611,0.101018,0.040469,0.004832,0.074166,0.004203,0.005304,0.000509,0.009810,0.005966,0.000395,0.000760,1.009461,0.081843,0.000347,0.004311,0.002332,0.003753,0.007584,0.006677,0.001142,0.005907,1,0.008427,0.008979,0.001854,0.009924,0.002247,0.006827,0.002837,1.005791,0.000571,0.000391,0.008351,0.008850,0.003180,0.003921,0.004654,0.184093,1.008394,0.004931,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.091505,0.021655,0.006815,0.068839,0.060492,0.003259,0.524568,0.056647,1.337179,0.009355,0.007426,0.134073,0.048367,0.010077,0.000530,0.006961,0.137014,1.001977,0.006328,0.139504,0.056653,0.305883,0.406768,CR,0.001878,0.645819,0.495308,0.006838,0.679257,0.001337,0.075868,0.103239,0.047454,0.006561,0.076510,0.001782,0.001422,0.008295,0.009362,0.005447,0.007345,0.004056,1.004291,0.081954,0.002709,0.007139,0.008358,0.007381,0.005901,0.001185,0.008013,0.008882,1,0.007327,0.002016,0.008686,0.008446,0.007794,0.009820,0.005080,1.005801,0.007425,0.009234,0.002471,0.009769,0.005433,0.001264,0.019176,0.154837,1.009307,0.009123,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.002455,0.013683,0.001373,0.055630,0.166782,0.009918,0.530929,0.049253,1.339909,0.006782,0.003515,0.134437,0.030063,0.009667,0.000783,0.008706,0.129017,0.704016,0.004980,0.138100,0.012498,0.273553,0.405175,CR,0.005899,0.654358,0.508670,0.008183,0.515282,0.008716,0.150209,0.206394,0.031705,0.009559,0.071547,0.005595,0.006363,0.005153,0.004876,0.001888,0.004961,0.006969,1.004728,0.060634,0.009982,0.008690,0.007364,0.008802,0.002520,0.003324,0.009455,0.008348,1,0.007053,0.003909,0.002478,0.006614,0.007686,0.000458,0.007320,1.007036,0.000664,0.003200,0.008507,0.004858,0.000063,0.002729,0.011720,0.153939,1.001671,0.002409,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.002483,0.015193,0.007605,0.038862,0.143630,0.006667,0.529305,0.048918,1.341735,0.000519,0.001362,0.121518,0.054221,0.009484,0.006698,0.003846,0.129539,0.917133,0.001653,0.126443,0.027897,0.233103,0.487460,CR,0.009479,0.650112,0.216507,0.008605,0.507712,0.006821,0.096441,0.106020,0.032733,0.008156,0.074432,0.004933,0.004831,0.007338,0.007447,0.006111,0.002246,0.001770,1.000904,0.062492,0.005860,0.007816,0.002470,0.007166,0.000155,0.001504,0.002019,0.002678,1,0.007728,0.003432,0.002199,0.005511,0.009656,0.003341,0.000264,1.002915,0.003079,0.003845,0.007190,0.002983,0.000535,0.009998,0.017598,0.120717,1.009886,0.004462,0


In [ ]:
print("New dataframe dimensions:",train_new.shape)
train_new.info()

New dataframe dimensions: (100000, 74)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 74 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   customer_ID  100000 non-null  object 
 1   S_2          100000 non-null  object 
 2   D_39         100000 non-null  float64
 3   B_1          100000 non-null  float64
 4   R_1          100000 non-null  float64
 5   B_4          100000 non-null  float64
 6   B_5          100000 non-null  float64
 7   R_2          100000 non-null  float64
 8   D_47         100000 non-null  float64
 9   B_7          100000 non-null  float64
 10  D_51         100000 non-null  float64
 11  B_9          100000 non-null  float64
 12  R_3          100000 non-null  float64
 13  B_10         100000 non-null  float64
 14  S_5          100000 non-null  float64
 15  B_11         100000 non-null  float64
 16  S_6          100000 non-null  float64
 17  R_4          100000 non-null 

# *Feature* *Engineering*

### TimeStamp Encoding



S_2 time and dat feature can be turned into a useful feature. By engineering into a week, month feature. It can help determin wether customers defualt based on

In [ ]:
train_new['S_2'] = pd.to_datetime(train_new['S_2'],format =  '%Y-%m-%d', errors = 'coerce' )
train_new['pay_week'] = train_new['S_2'].dt.week




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [ ]:
train_new['pay_week']

0        10
1        14
2        21
3        24
4        28
         ..
99995     1
99996     9
99997    10
99998    13
99999    17
Name: pay_week, Length: 100000, dtype: int64

In [ ]:
from math import ceil

def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """

    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))

train_new['Week'] = train_new['S_2'].apply(week_of_month)

In [ ]:
train_new.head()

,customer_ID,S_2,D_39,B_1,R_1,B_4,B_5,R_2,D_47,B_7,D_51,B_9,R_3,B_10,S_5,B_11,S_6,R_4,B_12,S_8,R_5,D_58,B_14,D_60,S_11,D_63,D_65,B_18,S_12,R_6,S_13,B_21,D_71,S_15,B_23,P_4,D_75,B_24,R_7,R_8,S_16,R_10,R_11,S_17,R_12,B_28,R_13,R_14,R_15,R_16,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_21,R_22,R_23,D_92,D_93,D_94,R_24,R_25,D_96,B_36,B_37,B_40,D_127,R_28,target,pay_week,Week
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.001733,0.008724,0.009228,0.080986,0.170600,0.006204,0.525351,0.059416,1.335856,0.008207,0.001423,0.096219,0.023381,0.002768,0.008322,0.008298,0.148266,0.922998,0.001882,0.158612,0.018385,0.199617,0.401619,CR,0.007126,0.652984,0.272008,0.008363,0.515222,0.002644,0.119403,0.108271,0.050882,0.007554,0.069067,0.004327,0.007562,0.001434,0.002271,0.007121,0.002456,0.008033,1.009825,0.084683,0.003820,0.000438,0.006452,0.005055,0.005720,0.007084,0.000198,0.008907,1,0.002537,0.005177,0.006626,0.009705,0.002450,0.007479,0.006893,1.006133,0.003569,0.008871,0.003950,0.003647,0.004950,0.009968,0.004572,0.210060,1.003319,0.001535,0,10,2
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.005775,0.004923,0.006151,0.069419,0.113239,0.006206,0.521311,0.057744,1.339794,0.008373,0.001984,0.099804,0.030599,0.002749,0.002482,0.005136,0.143530,0.919414,0.001610,0.148459,0.013035,0.151387,0.406326,CR,0.002413,0.647093,0.188970,0.004030,0.509048,0.004193,0.140611,0.101018,0.040469,0.004832,0.074166,0.004203,0.005304,0.000509,0.009810,0.005966,0.000395,0.000760,1.009461,0.081843,0.000347,0.004311,0.002332,0.003753,0.007584,0.006677,0.001142,0.005907,1,0.008427,0.008979,0.001854,0.009924,0.002247,0.006827,0.002837,1.005791,0.000571,0.000391,0.008351,0.008850,0.003180,0.003921,0.004654,0.184093,1.008394,0.004931,0,14,2
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.091505,0.021655,0.006815,0.068839,0.060492,0.003259,0.524568,0.056647,1.337179,0.009355,0.007426,0.134073,0.048367,0.010077,0.000530,0.006961,0.137014,1.001977,0.006328,0.139504,0.056653,0.305883,0.406768,CR,0.001878,0.645819,0.495308,0.006838,0.679257,0.001337,0.075868,0.103239,0.047454,0.006561,0.076510,0.001782,0.001422,0.008295,0.009362,0.005447,0.007345,0.004056,1.004291,0.081954,0.002709,0.007139,0.008358,0.007381,0.005901,0.001185,0.008013,0.008882,1,0.007327,0.002016,0.008686,0.008446,0.007794,0.009820,0.005080,1.005801,0.007425,0.009234,0.002471,0.009769,0.005433,0.001264,0.019176,0.154837,1.009307,0.009123,0,21,4
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.002455,0.013683,0.001373,0.055630,0.166782,0.009918,0.530929,0.049253,1.339909,0.006782,0.003515,0.134437,0.030063,0.009667,0.000783,0.008706,0.129017,0.704016,0.004980,0.138100,0.012498,0.273553,0.405175,CR,0.005899,0.654358,0.508670,0.008183,0.515282,0.008716,0.150209,0.206394,0.031705,0.009559,0.071547,0.005595,0.006363,0.005153,0.004876,0.001888,0.004961,0.006969,1.004728,0.060634,0.009982,0.008690,0.007364,0.008802,0.002520,0.003324,0.009455,0.008348,1,0.007053,0.003909,0.002478,0.006614,0.007686,0.000458,0.007320,1.007036,0.000664,0.003200,0.008507,0.004858,0.000063,0.002729,0.011720,0.153939,1.001671,0.002409,0,24,3
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.002483,0.015193,0.007605,0.038862,0.143630,0.006667,0.529305,0.048918,1.341735,0.000519,0.001362,0.121518,0.054221,0.009484,0.006698,0.003846,0.129539,0.917133,0.001653,0.126443,0.027897,0.233103,0.487460,CR,0.009479,0.650112,0.216507,0.008605,0.507712,0.006821,0.096441,0.106020,0.032733,0.008156,0.074432,0.004933,0.004831,0.007338,0.007447,0.006111,0.002246,0.001770,1.000904,0.062492,0.005860,0.007816,0.002470,0.007166,0.000155,0.001504,0.002019,0.002678,1,0.007728,0.003432,0.002199,0.005511,0.009656,0.003341,0.000264,1.002915,0.003079,0.003845,0.007190,0.002983,0.000535,0.009998,0.017598,0.120717,1.009886,0.004462,0,28,3


### Target Encoding


For the 'D_63' it is a categorical feature with distinct categorical values as printed below

In [ ]:
train['D_63'].unique()

array(['CR', 'CO', 'CL', 'XZ', 'XM', 'XL'], dtype=object)

For optimized machine learning models. it is better to get the data into numerical data types and not categorical. This can be done through encoding.There are 2 methods of encoding that will be utilized. Firstly Ordinal Encoding and then One Hot Encoding

#### Ordinal Encoding

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
train_copy = train_new.drop(['customer_ID'], axis=1)
s = (train_copy.dtypes == 'object')
object_cols = list(s[s].index)
# Make copy to avoid changing original data 
label_X_train = train_copy.copy()
#label_X_valid = X_valid.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(train_copy[object_cols])
#label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

In [ ]:
label_X_train

,S_2,D_39,B_1,R_1,B_4,B_5,R_2,D_47,B_7,D_51,B_9,R_3,B_10,S_5,B_11,S_6,R_4,B_12,S_8,R_5,D_58,B_14,D_60,S_11,D_63,D_65,B_18,S_12,R_6,S_13,B_21,D_71,S_15,B_23,P_4,D_75,B_24,R_7,R_8,S_16,R_10,R_11,S_17,R_12,B_28,R_13,R_14,R_15,R_16,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_21,R_22,R_23,D_92,D_93,D_94,R_24,R_25,D_96,B_36,B_37,B_40,D_127,R_28,target,pay_week,Week
0,2017-03-09,0.001733,0.008724,0.009228,0.080986,0.170600,0.006204,0.525351,0.059416,1.335856,0.008207,0.001423,0.096219,0.023381,0.002768,0.008322,0.008298,0.148266,0.922998,0.001882,0.158612,0.018385,0.199617,0.401619,2.0,0.007126,0.652984,0.272008,0.008363,0.515222,0.002644,0.119403,0.108271,0.050882,0.007554,0.069067,0.004327,0.007562,0.001434,0.002271,0.007121,0.002456,0.008033,1.009825,0.084683,0.003820,0.000438,0.006452,0.005055,0.005720,0.007084,0.000198,0.008907,1,0.002537,0.005177,0.006626,0.009705,0.002450,0.007479,0.006893,1.006133,0.003569,0.008871,0.003950,0.003647,0.004950,0.009968,0.004572,0.210060,1.003319,0.001535,0,10,2
1,2017-04-07,0.005775,0.004923,0.006151,0.069419,0.113239,0.006206,0.521311,0.057744,1.339794,0.008373,0.001984,0.099804,0.030599,0.002749,0.002482,0.005136,0.143530,0.919414,0.001610,0.148459,0.013035,0.151387,0.406326,2.0,0.002413,0.647093,0.188970,0.004030,0.509048,0.004193,0.140611,0.101018,0.040469,0.004832,0.074166,0.004203,0.005304,0.000509,0.009810,0.005966,0.000395,0.000760,1.009461,0.081843,0.000347,0.004311,0.002332,0.003753,0.007584,0.006677,0.001142,0.005907,1,0.008427,0.008979,0.001854,0.009924,0.002247,0.006827,0.002837,1.005791,0.000571,0.000391,0.008351,0.008850,0.003180,0.003921,0.004654,0.184093,1.008394,0.004931,0,14,2
2,2017-05-28,0.091505,0.021655,0.006815,0.068839,0.060492,0.003259,0.524568,0.056647,1.337179,0.009355,0.007426,0.134073,0.048367,0.010077,0.000530,0.006961,0.137014,1.001977,0.006328,0.139504,0.056653,0.305883,0.406768,2.0,0.001878,0.645819,0.495308,0.006838,0.679257,0.001337,0.075868,0.103239,0.047454,0.006561,0.076510,0.001782,0.001422,0.008295,0.009362,0.005447,0.007345,0.004056,1.004291,0.081954,0.002709,0.007139,0.008358,0.007381,0.005901,0.001185,0.008013,0.008882,1,0.007327,0.002016,0.008686,0.008446,0.007794,0.009820,0.005080,1.005801,0.007425,0.009234,0.002471,0.009769,0.005433,0.001264,0.019176,0.154837,1.009307,0.009123,0,21,4
3,2017-06-13,0.002455,0.013683,0.001373,0.055630,0.166782,0.009918,0.530929,0.049253,1.339909,0.006782,0.003515,0.134437,0.030063,0.009667,0.000783,0.008706,0.129017,0.704016,0.004980,0.138100,0.012498,0.273553,0.405175,2.0,0.005899,0.654358,0.508670,0.008183,0.515282,0.008716,0.150209,0.206394,0.031705,0.009559,0.071547,0.005595,0.006363,0.005153,0.004876,0.001888,0.004961,0.006969,1.004728,0.060634,0.009982,0.008690,0.007364,0.008802,0.002520,0.003324,0.009455,0.008348,1,0.007053,0.003909,0.002478,0.006614,0.007686,0.000458,0.007320,1.007036,0.000664,0.003200,0.008507,0.004858,0.000063,0.002729,0.011720,0.153939,1.001671,0.002409,0,24,3
4,2017-07-16,0.002483,0.015193,0.007605,0.038862,0.143630,0.006667,0.529305,0.048918,1.341735,0.000519,0.001362,0.121518,0.054221,0.009484,0.006698,0.003846,0.129539,0.917133,0.001653,0.126443,0.027897,0.233103,0.487460,2.0,0.009479,0.650112,0.216507,0.008605,0.507712,0.006821,0.096441,0.106020,0.032733,0.008156,0.074432,0.004933,0.004831,0.007338,0.007447,0.006111,0.002246,0.001770,1.000904,0.062492,0.005860,0.007816,0.002470,0.007166,0.000155,0.001504,0.002019,0.002678,1,0.007728,0.003432,0.002199,0.005511,0.009656,0.003341,0.000264,1.002915,0.003079,0.003845,0.007190,0.002983,0.000535,0.009998,0.017598,0.120717,1.009886,0.004462,0,28,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2018-01-02,0.124405,0.597829,0.004703,0.360951,0.011009,0.002830,0.418322,0.521920,0.00264

#### One Hot Encoding


In [ ]:

from sklearn.preprocessing import OneHotEncoder
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train_copy[object_cols]))
#OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = train_copy.index
#OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = train_copy.drop(object_cols, axis=1)
#num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
#OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)


### Class Balancing


In [ ]:
import sklearn
X = OH_X_train.drop(['S_2','target'],axis =1)
y = labels["target"]


In [ ]:
# define categorical variables (columns)
categorical = list(X.select_dtypes('object').columns)
print(f"Categorical variables (columns) are: {categorical}")

# define numerical variables (columns)
numerical = X.select_dtypes(include="object").columns
print(f"Numerical variables (columns) are: {numerical}")

Categorical variables (columns) are: []
Numerical variables (columns) are: Index([], dtype='object')


In [ ]:
# define categorical pipeline
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent', missing_values=np.nan)),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
    ('scaler', StandardScaler())
])

print(cat_pipe)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder',
                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                unknown_value=-1)),
                ('scaler', StandardScaler())])


In [ ]:
# define numerical pipeline
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent', missing_values=np.nan)),
    ('scaler', StandardScaler())
])
print(num_pipe)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('scaler', StandardScaler())])


In [ ]:
preprocess = ColumnTransformer([('num', num_pipe, numerical)])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state = 42)

In [ ]:
X_train

,D_39,B_1,R_1,B_4,B_5,R_2,D_47,B_7,D_51,B_9,R_3,B_10,S_5,B_11,S_6,R_4,B_12,S_8,R_5,D_58,B_14,D_60,S_11,D_65,B_18,S_12,R_6,S_13,B_21,D_71,S_15,B_23,P_4,D_75,B_24,R_7,R_8,S_16,R_10,R_11,S_17,R_12,B_28,R_13,R_14,R_15,R_16,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_21,R_22,R_23,D_92,D_93,D_94,R_24,R_25,D_96,B_36,B_37,B_40,D_127,R_28,pay_week,Week,0,1,2,3,4,5
69504,0.473433,0.143935,0.004391,0.264498,0.028456,0.004374,0.311321,0.208351,0.008962,0.215360,0.002485,0.041565,0.013358,0.104286,0.005848,0.000721,0.014327,0.005260,0.009711,0.410123,0.050590,0.289917,0.485667,0.001596,0.203464,0.175576,0.004073,0.007752,0.000913,0.010163,0.501190,0.172964,0.003699,0.266733,0.000979,0.003617,0.009274,0.005382,0.009523,0.001967,1.157312,1.004747,0.087072,0.001143,0.009970,0.009289,0.005193,0.000196,0.007059,0.005540,0.003172,1,0.009371,0.009740,0.004773,0.002877,0.006069,0.003657,0.004709,0.008883,0.007395,0.006932,0.001603,0.008453,0.002150,0.002576,0.137608,0.243832,0.004547,0.001345,13,5,0.0,1.0,0.0,0.0,0.0,0.0
87161,0.211142,0.104378,0.006885,0.044113,0.008058,0.001988,0.392015,0.097390,0.006833,0.536882,0.005258,0.125320,0.046715,0.068756,0.000179,0.006712,0.012645,0.602921,0.006741,0.007532,0.024432,0.726553,0.327974,0.004622,0.449695,0.296127,0.007763,0.288384,0.009167,0.009349,0.205133,0.062286,0.932586,0.066707,0.008953,0.008786,0.006584,0.000888,0.007465,0.006577,0.004383,1.002099,0.013180,0.002769,0.006592,0.009137,0.006191,0.006459,0.008535,0.000539,0.003515,1,0.002855,0.007242,0.007290,0.004528,0.004056,0.000262,0.000843,0.005836,0.005543,0.007755,0.001426,0.009708,0.005192,0.003222,0.103211,0.058806,0.002849,0.001581,25,4,0.0,1.0,0.0,0.0,0.0,0.0
53876,0.001908,0.163646,0.002611,0.094137,0.009123,0.005966,0.239982,0.112396,0.005171,0.637767,0.200675,0.022407,0.009068,0.122126,0.005356,0.002222,0.013644,0.121281,0.007263,0.233417,0.041015,0.579947,0.563919,0.005224,0.214172,0.193258,0.000079,0.007501,0.002934,0.006872,0.400344,0.079942,0.004730,0.271376,0.007021,0.003229,0.003782,0.003664,0.007099,0.002628,0.007461,1.009987,0.033496,0.004835,0.006619,0.009718,0.002246,0.007903,0.007353,0.005436,0.009427,1,0.001000,0.003667,0.005040,0.006797,0.002344,0.002204,0.008719,0.009393,0.009100,0.007850,0.009191,0.009838,0.007173,0.001628,0.164028,0.109424,0.003159,0.009087,9,1,0.0,1.0,0.0,0.0,0.0,0.0
42572,0.007886,0.010047,0.000585,0.020781,0.805670,0.001944,0.416899,0.006788,1.334976,0.065816,0.000651,0.296998,0.001734,0.004753,0.008531,0.001733,1.487781,1.002512,0.004433,0.009363,0.071993,0.862236,0.448329,0.002350,1.008684,0.164365,0.000392,0.687757,0.004979,0.007456,0.103786,0.004279,0.003647,0.002660,0.000328,0.002865,0.005730,0.000798,0.002983,0.502944,0.004670,1.003281,0.059053,0.001269,0.005238,0.002154,0.001351,0.009907,0.002087,0.002289,0.002617,1,0.002751,0.001701,0.006638,0.002806,0.001073,0.005083,0.001862,1.005382,0.002985,0.006531,0.002071,0.001881,1.008052,0.000237,0.011151,0.025812,1.006868,0.007139,41,3,0.0,1.0,0.0,0.0,0.0,0.0
84558,0.241019,0.007850,0.004421,0.133504,0.001468,0.009809,0.605534,0.062154,0.341719,0.003563,0.008674,0.114623,0.008162,0.000416,1.007114,0.003076,0.013909,0.002439,0.002769,0.114485,0.008613,0.001497,0.283581,0.000202,0.649202,0.191735,0.000871,0.000343,0.005619,0.016580,0.506605,0.043451,0.008578,0.068398,0.008037,0.003502,0.000616,0.008921,0.000759,0.005436,0.008168,1.009081,0.077617,0.005854,0.002453,0.006227,0.000073,0.005554,0.008740,0.005737,0.000601,1,0.004812,0.008699,0.006715,0.008652,0.007920,0.003391,0.008820,0.004149,0.008504,0.007635,0.004985,0.000395,0.003878,0.006305,0.008352,0.028648,0.005849,0.003718,48,1,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73734,0.004600,0.010969,0.006830,0

In [ ]:
from collections import Counter
counter = Counter(y_train)
print(counter)

Counter({0: 59167, 1: 20833})


In [ ]:
over = SMOTE(random_state = 42)
under = RandomUnderSampler(random_state = 42)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
X_train, y_train = pipeline.fit_resample(X_train, y_train)
counter = Counter(y_train)
print(counter)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


Counter({0: 59167, 1: 59167})


### Mutual Information

There is a large number of features, some may have little or no effect on the result so we shall explore the mutuual 

In [ ]:
import sklearn
X = X_train
y = y_train


# Label encoding for categoricals
#for colname in X.select_dtypes("object"):
 #   X[colname], _ = X[colname].factorize()

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X.dtypes == int


def make_mi_scores(X, y, discrete_features):
    mi_scores = sklearn.feature_selection.mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
   

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

print_full(mi_scores)

S_15        0.146171
S_11        0.146106
R_3         0.139622
S_8         0.132181
D_75        0.129595
S_13        0.113071
D_39        0.099074
B_18        0.091814
D_51        0.078095
B_4         0.045549
R_16        0.033494
R_11        0.032510
R_1         0.026095
R_10        0.019659
D_86        0.018915
P_4         0.018848
D_65        0.018400
D_96        0.017951
D_92        0.017195
D_127       0.016211
R_5         0.015907
S_6         0.014842
S_18        0.014663
R_15        0.013734
B_31        0.013531
R_17        0.013195
D_94        0.013054
R_19        0.012947
R_22        0.012909
B_32        0.012604
R_8         0.012459
R_2         0.012270
R_12        0.011785
R_21        0.011720
R_13        0.011477
D_93        0.011260
S_20        0.011186
R_4         0.011028
R_7         0.011020
B_36        0.010967
R_25        0.010742
R_24        0.010445
R_14        0.009435
B_24        0.009277
S_19        0.009213
R_18        0.009154
S_17        0.008869
S_16        0

The least possible mutual information between quantities is 0.0. When MI is zero, the quantities are independent: neither can tell you anything about the other. There is alot of features that have a value of 0. MOst valuuse are close to zero, it shows that the features are not so dependent on the defaulted or not defaulted target.

In [ ]:
mi_scores[mi_scores>0.0007]
X_train

,D_39,B_1,R_1,B_4,B_5,R_2,D_47,B_7,D_51,B_9,R_3,B_10,S_5,B_11,S_6,R_4,B_12,S_8,R_5,D_58,B_14,D_60,S_11,D_65,B_18,S_12,R_6,S_13,B_21,D_71,S_15,B_23,P_4,D_75,B_24,R_7,R_8,S_16,R_10,R_11,S_17,R_12,B_28,R_13,R_14,R_15,R_16,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_21,R_22,R_23,D_92,D_93,D_94,R_24,R_25,D_96,B_36,B_37,B_40,D_127,R_28,pay_week,Week,0,1,2,3,4,5
0,0.473433,0.143935,0.004391,0.264498,0.028456,0.004374,0.311321,0.208351,0.008962,0.215360,0.002485,0.041565,0.013358,0.104286,0.005848,0.000721,0.014327,0.005260,0.009711,0.410123,0.050590,0.289917,0.485667,0.001596,0.203464,0.175576,0.004073,0.007752,0.000913,0.010163,0.501190,0.172964,0.003699,0.266733,0.000979,0.003617,0.009274,0.005382,0.009523,0.001967,1.157312,1.004747,0.087072,0.001143,0.009970,0.009289,0.005193,0.000196,0.007059,0.005540,0.003172,1,0.009371,0.009740,0.004773,0.002877,0.006069,0.003657,0.004709,0.008883,0.007395,0.006932,0.001603,0.008453,0.002150,0.002576,0.137608,0.243832,0.004547,0.001345,13,5,0.0,1.0,0.0,0.0,0.0,0.0
1,0.211142,0.104378,0.006885,0.044113,0.008058,0.001988,0.392015,0.097390,0.006833,0.536882,0.005258,0.125320,0.046715,0.068756,0.000179,0.006712,0.012645,0.602921,0.006741,0.007532,0.024432,0.726553,0.327974,0.004622,0.449695,0.296127,0.007763,0.288384,0.009167,0.009349,0.205133,0.062286,0.932586,0.066707,0.008953,0.008786,0.006584,0.000888,0.007465,0.006577,0.004383,1.002099,0.013180,0.002769,0.006592,0.009137,0.006191,0.006459,0.008535,0.000539,0.003515,1,0.002855,0.007242,0.007290,0.004528,0.004056,0.000262,0.000843,0.005836,0.005543,0.007755,0.001426,0.009708,0.005192,0.003222,0.103211,0.058806,0.002849,0.001581,25,4,0.0,1.0,0.0,0.0,0.0,0.0
2,0.001908,0.163646,0.002611,0.094137,0.009123,0.005966,0.239982,0.112396,0.005171,0.637767,0.200675,0.022407,0.009068,0.122126,0.005356,0.002222,0.013644,0.121281,0.007263,0.233417,0.041015,0.579947,0.563919,0.005224,0.214172,0.193258,0.000079,0.007501,0.002934,0.006872,0.400344,0.079942,0.004730,0.271376,0.007021,0.003229,0.003782,0.003664,0.007099,0.002628,0.007461,1.009987,0.033496,0.004835,0.006619,0.009718,0.002246,0.007903,0.007353,0.005436,0.009427,1,0.001000,0.003667,0.005040,0.006797,0.002344,0.002204,0.008719,0.009393,0.009100,0.007850,0.009191,0.009838,0.007173,0.001628,0.164028,0.109424,0.003159,0.009087,9,1,0.0,1.0,0.0,0.0,0.0,0.0
3,0.007886,0.010047,0.000585,0.020781,0.805670,0.001944,0.416899,0.006788,1.334976,0.065816,0.000651,0.296998,0.001734,0.004753,0.008531,0.001733,1.487781,1.002512,0.004433,0.009363,0.071993,0.862236,0.448329,0.002350,1.008684,0.164365,0.000392,0.687757,0.004979,0.007456,0.103786,0.004279,0.003647,0.002660,0.000328,0.002865,0.005730,0.000798,0.002983,0.502944,0.004670,1.003281,0.059053,0.001269,0.005238,0.002154,0.001351,0.009907,0.002087,0.002289,0.002617,1,0.002751,0.001701,0.006638,0.002806,0.001073,0.005083,0.001862,1.005382,0.002985,0.006531,0.002071,0.001881,1.008052,0.000237,0.011151,0.025812,1.006868,0.007139,41,3,0.0,1.0,0.0,0.0,0.0,0.0
4,0.241019,0.007850,0.004421,0.133504,0.001468,0.009809,0.605534,0.062154,0.341719,0.003563,0.008674,0.114623,0.008162,0.000416,1.007114,0.003076,0.013909,0.002439,0.002769,0.114485,0.008613,0.001497,0.283581,0.000202,0.649202,0.191735,0.000871,0.000343,0.005619,0.016580,0.506605,0.043451,0.008578,0.068398,0.008037,0.003502,0.000616,0.008921,0.000759,0.005436,0.008168,1.009081,0.077617,0.005854,0.002453,0.006227,0.000073,0.005554,0.008740,0.005737,0.000601,1,0.004812,0.008699,0.006715,0.008652,0.007920,0.003391,0.008820,0.004149,0.008504,0.007635,0.004985,0.000395,0.003878,0.006305,0.008352,0.028648,0.005849,0.003718,48,1,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118329,0.190226,0.201237,0.004919,0.199598,0.041359,0.

In [ ]:
X_train = X_train[mi_scores[mi_scores>0.00007].index]
X_test = X_test[mi_scores[mi_scores>0.00007].index]
y = y


In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state = 42)
print("Shape of X_train", X_train.shape)
print("Shape of X_test", X_test.shape)
print("Shape of y_train", y_train.shape)
print("Shape of y_test", y_test.shape)

Shape of X_train (118334, 73)
Shape of X_test (20000, 73)
Shape of y_train (118334,)
Shape of y_test (20000,)


# ML Model Score functions

In [ ]:
# Amex Evaluation Metric for reference
# please refer sample notebook provided under competition page for details
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
    
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()
    
    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [ ]:
# Look at confusion matrix 

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.grid(None)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
scoring = ['accuracy', 'precision', 'recall','f1','roc_auc']
cv = StratifiedKFold(n_splits=10, shuffle = True, random_state = 42)

# ML Model


In [ ]:
algo=[
    [KNeighborsClassifier(), 'KNeighborsClassifier'], 
    [LogisticRegression(), 'LogisticRegression'], 
    [DecisionTreeClassifier(), 'DecisionTreeClassifier'],
    [GradientBoostingClassifier(), 'GradientBoostingClassifier'],
    [RandomForestClassifier(), 'RandomForestClassifier'],
    [AdaBoostClassifier(), 'AdaBoostClassifier'],
    [GaussianNB(), 'GaussianNB'],
    [XGBClassifier(), "XGBClassifier"],
    [LGBMClassifier(), "LGBMClassifier"],
    [CatBoostClassifier(), "CatBoostClassifier"]
]
import time
model_scores=[]
for a in algo:
    st = time.time()
    model = a[0]
    print(model)
    scores = cross_validate(model, X_train, y_train, scoring=scoring, cv=cv, n_jobs=-1, return_train_score=True,return_estimator=True)
    et = time.time()
    elapsed = et - st
    print('Training Score: Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f},f1-score: {:.2f}, ROC AUC: {:.2f}'.format(np.mean(scores['train_accuracy']),np.mean(scores['train_precision']), np.mean(scores['train_recall']), np.mean(scores['train_f1']), np.mean(scores['train_roc_auc'])))
    print('Validation Score: Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f},f1-score: {:.2f}, ROC AUC: {:.2f}'.format(np.mean(scores['test_accuracy']),np.mean(scores['test_precision']), np.mean(scores['test_recall']), np.mean(scores['test_f1']), np.mean(scores['test_roc_auc'])))
    model_scores.append([np.mean(scores['test_accuracy']),[elapsed/60], a[1]])

print(model_scores)
print(f'best score = {max(model_scores)}')   

KNeighborsClassifier()
Training Score: Accuracy: 0.79, Precision: 0.75, Recall: 0.87,f1-score: 0.81, ROC AUC: 0.88
Validation Score: Accuracy: 0.66, Precision: 0.63, Recall: 0.76,f1-score: 0.69, ROC AUC: 0.72
LogisticRegression()
Training Score: Accuracy: 0.53, Precision: 0.52, Recall: 0.64,f1-score: 0.58, ROC AUC: 0.54
Validation Score: Accuracy: 0.53, Precision: 0.52, Recall: 0.64,f1-score: 0.57, ROC AUC: 0.53
DecisionTreeClassifier()
Training Score: Accuracy: 1.00, Precision: 1.00, Recall: 1.00,f1-score: 1.00, ROC AUC: 1.00
Validation Score: Accuracy: 0.70, Precision: 0.69, Recall: 0.72,f1-score: 0.70, ROC AUC: 0.70
GradientBoostingClassifier()


In [ ]:
dscore=pd.DataFrame(model_scores, columns=['score', 'time [min]', 'classifier'])
dscore.sort_values('score', ascending=False)

# Model Optimization (Voting)

In [ ]:
# import voting classifier
from sklearn.ensemble import VotingClassifier

# create a voting classifier with hard voting
voting_classifier_hard = VotingClassifier(
    estimators = [('rfc',RandomForestClassifier()),
                  ('lgb', LGBMClassifier()),
                  ('gnb', GaussianNB())], 
    voting='hard')

# create a voting classifier with soft voting
voting_classifier_soft = VotingClassifier(
    estimators = [('rfc',RandomForestClassifier()),
                  ('lgb', LGBMClassifier()),
                  ('gnb', GaussianNB())], 
    voting='soft')

# make predictions with the hard voting model
voting_classifier_hard.fit(X_train, y_train)
y_pred_vch = voting_classifier_hard.predict(X_test)

# make predictions with the soft voting model
voting_classifier_soft.fit(X_train, y_train)
y_pred_vcs = voting_classifier_soft.predict(X_test)

# evaluate both models with the f-1 score
f1_vch = f1_score(y_test, y_pred_vch)
f1_vcs = f1_score(y_test, y_pred_vcs)

# print the f-1 scores
print('F1-score of the hard voting classifier: {}'.format(np.round(f1_vch,4)))
print('F1-score of the soft voting classifier: {}'.format(np.round(f1_vcs,4)))

# Model Optimization (PCA)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(.95)
pca.fit(X_train)
PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [ ]:
pca.n_components_

In [ ]:
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
algo=[
    [KNeighborsClassifier(), 'KNeighborsClassifier'], 
    [LogisticRegression(), 'LogisticRegression'], 
    [DecisionTreeClassifier(), 'DecisionTreeClassifier'],
    [GradientBoostingClassifier(), 'GradientBoostingClassifier'],
    [RandomForestClassifier(), 'RandomForestClassifier'],
    [AdaBoostClassifier(), 'AdaBoostClassifier'],
    [GaussianNB(), 'GaussianNB'],
    [XGBClassifier(), "XGBClassifier"],
    [LGBMClassifier(), "LGBMClassifier"],
    [CatBoostClassifier(), "CatBoostClassifier"]
]

model_scores=[]
for a in algo:
    st = time.time()
    model = a[0]
    print(model)
    scores = cross_validate(model, X_train, y_train, scoring=scoring, cv=cv, n_jobs=-1, return_train_score=True,return_estimator=True)
    et = time.time()
    elapsed = et - st
    print('Training Score: Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f},f1-score: {:.2f}, ROC AUC: {:.2f}'.format(np.mean(scores['train_accuracy']),np.mean(scores['train_precision']), np.mean(scores['train_recall']), np.mean(scores['train_f1']), np.mean(scores['train_roc_auc'])))
    print('Validation Score: Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f},f1-score: {:.2f}, ROC AUC: {:.2f}'.format(np.mean(scores['test_accuracy']),np.mean(scores['test_precision']), np.mean(scores['test_recall']), np.mean(scores['test_f1']), np.mean(scores['test_roc_auc'])))
    model_scores.append([np.mean(scores['test_accuracy']),[elapsed/60], a[1]])

print(model_scores)
print(f'best score = {max(model_scores)}')   

In [ ]:
dscore=pd.DataFrame(model_scores, columns=['score', 'time [min]', 'classifier'])
dscore.sort_values('score', ascending=False)